<a href="https://colab.research.google.com/github/bhaveshasasik/nfl_game_predictor/blob/main/safety_impact_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.preprocessing import MinMaxScaler
import os

In [ ]:
data = pd.read_csv('sample_data/safety_stats.csv')

data = data[data["Pos"] == "S"]

#print(data.columns)
print(data.head())

    Rk            Player  Age Team Pos   G  GS  Int  Yds  IntTD  ...  FR  \
3    4  Jessie Bates III   27  ATL   S  11  11    2    0      0  ...   1   
4    5         Vonn Bell   30  CIN   S  11  11    1   32      0  ...   0   
8    9      Jeremy Chinn   26  WAS   S  11  11    1    0      0  ...   2   
9   10       Andre Cisco   24  JAX   S  11  11    1   22      0  ...   0   
14  15        Nick Cross   23  IND   S  11  11    2   20      0  ...   0   

    Yds.1  FRTD   Sk  Comb  Solo  Ast  TFL  QBHits  Sfty  
3       0     0  0.0    70    42   28    1       0     0  
4       0     0  0.0    48    20   28    1       1     0  
8       0     0  0.0    80    51   29    3       0     0  
9       0     0  0.0    51    33   18    0       0     0  
14      0     0  0.0   102    63   39    2       0     0  

[5 rows x 24 columns]


In [ ]:
data["Interceptions_Per_Game"] = data["Int"] / data["G"]
data["Passing_Defended_Per_Game"] = data["PD"] / data["G"]
data["Forced_Fumbles_Per_Game"] = data["FF"] / data["G"]
data["Defensive_Touchdowns_Per_Game"] = data["IntTD"] / data["G"]
data["Yards_From_Interceptions"] = data["Yds"] + data["Yds.1"] / data["G"]
data["Solo_Per_Game"] = data["Solo"] / data["G"]
data["Ast_Per_Game"] = data["Ast"] / data["G"]
data["TFL_Per_Game"] = data["TFL"] / data["G"]
data["QBHits_Per_Game"] = data["QBHits"] / data["G"]

In [ ]:
metrics = ["Interceptions_Per_Game", "Passing_Defended_Per_Game", "Forced_Fumbles_Per_Game", "Defensive_Touchdowns_Per_Game", "Yards_From_Interceptions", "Solo_Per_Game", "Ast_Per_Game", "TFL_Per_Game", "QBHits_Per_Game"]

In [ ]:
scaler = MinMaxScaler()
data[metrics] = scaler.fit_transform(data[metrics])

In [ ]:
data['Safety_Impact_Score'] = (
    0.35 * data['Interceptions_Per_Game'] +
    0.2 * data['Passing_Defended_Per_Game'] +
    0.2 * (data['Solo_Per_Game'] + data['Ast_Per_Game']) +
    0.15 * data['Defensive_Touchdowns_Per_Game'] +
    0.1 * (data['TFL_Per_Game'] + data['QBHits_Per_Game'])
)


In [ ]:
top_players_per_team = (
    data.sort_values(by=['Team', 'Safety_Impact_Score'], ascending=[True, False])
    .groupby('Team')
    .head(1)
)

In [ ]:
print("Top Impactful Safeties per Team in 2023 based on Combined Impact Score:\n")
print(top_players_per_team[['Team', 'Player', 'Safety_Impact_Score']])

Top Impactful Safeties per Team in 2023 based on Combined Impact Score:

     Team                Player  Safety_Impact_Score
992   2TM       Terrell Edmunds             0.043433
212   ARI           Budda Baker             0.563993
3     ATL      Jessie Bates III             0.475609
24    BAL         Kyle Hamilton             0.547467
318   BUF           Taylor Rapp             0.485643
348   CAR          Xavier Woods             0.455471
592   CHI        Jonathan Owens             0.013560
61    CIN            Geno Stone             0.334381
633   CLE          Grant Delpit             0.373441
345   DAL        Donovan Wilson             0.389944
280   DEN         Brandon Jones             0.425226
284   DET          Kerby Joseph             0.895978
297   GNB       Xavier McKinney             0.701686
49    HOU           Jalen Pitre             0.537016
14    IND            Nick Cross             0.538200
776   JAX    Darnell Savage Jr.             0.313013
319   KAN           Justin